In [1]:
import pandas as pd
import numpy as np
import math 
import os 
from datetime import datetime, timedelta
import pymap3d 
import random
import pickle
from scipy.special import lambertw
from scipy import stats
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline
from bokeh.plotting import figure, show, output_notebook, output_file, reset_output, save
from bokeh.layouts import gridplot, row, column
from bokeh.models import Range1d
output_notebook() 


Loading BokehJS ...

# Offline

## Data

In [2]:
#loading the data
dataset="cabspotting" # OPTIONS ARE: cabspotting privamov
user="oilrag"#examples: abboip oilrag 51 90 14
local_url="datasets/"+dataset+"-tree/"+user+".csv"

Hp=4 #future steps
sol_file=user+'/databuild_Hp'+str(Hp)
exists = os.path.isfile(local_url)

if exists:
    data = pd.read_csv(local_url,names=["latitude","longitude","timestamp"])
    print("uploaded from local file")
else:
    print("data not found")

if not os.path.exists(sol_file):
    os.makedirs(sol_file)

data['time']=[datetime.utcfromtimestamp(tstamp/1000) for tstamp in data['timestamp']]
data['elapsedtime']=(data['timestamp']-data['timestamp'][0])/1000
whole_duration = data['elapsedtime'][len(data['timestamp'])-1]/60
print('Whole duration: '+str(math.floor(whole_duration))+' min')    
    
data=data.set_index('time').sort_index().copy()
ell_grs80 = pymap3d.Ellipsoid('grs80') 
data['altitude']=np.zeros(len(data.index))
lat0, lon0, h0 = data['latitude'][0],data['longitude'][0],data['altitude'][0]
data['x'], data['y'], data['u_enu'] = pymap3d.geodetic2enu(data['latitude'], data['longitude'], data['altitude'], lat0, lon0, h0, ell=ell_grs80)
print(data.head())

uploaded from local file
Whole duration: 1126 min
                     latitude  longitude      timestamp  elapsedtime  \
time                                                                   
2008-05-17 10:00:06  37.74574 -122.42685  1211018406000          0.0   
2008-05-17 10:01:17  37.74967 -122.42729  1211018477000         71.0   
2008-05-17 10:03:13  37.75233 -122.43419  1211018593000        187.0   
2008-05-17 10:04:03  37.75696 -122.43463  1211018643000        237.0   
2008-05-17 10:05:13  37.76122 -122.43501  1211018713000        307.0   

                     altitude           x            y     u_enu  
time                                                              
2008-05-17 10:00:06       0.0    0.000000     0.000000  0.000000  
2008-05-17 10:01:17       0.0  -38.777282   436.197503 -0.015077  
2008-05-17 10:03:13       0.0 -646.852405   731.460882 -0.074827  
2008-05-17 10:04:03       0.0 -685.585574  1245.356220 -0.158740  
2008-05-17 10:05:13       0.0 -719.030524  

Code for resampling the data: for every ts-long interval, $[i t_s, \ (i+1) t_s ]$, the mean of the transmitted data is computed and associated to the smpling instant $(i+1) t_s$.

In [3]:
# Real transmitted data
Tmin = 0
Tmax = 20000 #20000 oilrag #50000 abboip #86400 51
# Tmax = data['elapsedtime'][-1]
xr = data['x'][(data['elapsedtime'] >= Tmin) & (data['elapsedtime'] < Tmax)]
yr = data['y'][(data['elapsedtime'] >= Tmin) & (data['elapsedtime'] < Tmax)]
ur = data['u_enu'][(data['elapsedtime'] >= Tmin) & (data['elapsedtime'] < Tmax)]
Timer = data['elapsedtime'][(data['elapsedtime'] >= Tmin) & (data['elapsedtime'] < Tmax)]
print('transmitted positions',len(Timer))
# Resampling the data with sample time ts, averaging over the intervals
ts = 30  # Sampling time
tmax = Timer[-1]  # Final time
nt = int(tmax/ts)+1
print('resampling points',nt)

newx = []
newy = []
newu = []
newb = []
newTimer = []
newvel = []
xtmin1 =0
ytmin1 =0
tmin1 =-100000
for i in range(0,nt):
    xt = data['x'][(data['elapsedtime'] >= i*ts) & (data['elapsedtime'] < (i+1)*ts)].mean() 
    yt = data['y'][(data['elapsedtime'] >= i*ts) & (data['elapsedtime'] < (i+1)*ts)].mean() 
    ut = data['u_enu'][(data['elapsedtime'] >= i*ts) & (data['elapsedtime'] < (i+1)*ts)].mean() 
    if np.isnan(xt) == False:
        newx += [xt]
        newy += [yt]
        newu += [ut]
        newb += [1]
        newvel += [np.sqrt((xt-xtmin1)**2+(yt-ytmin1)**2)/((i+1)*ts-tmin1)]
        xtmin1 =xt
        ytmin1 =yt
        tmin1 =(i+1)*ts
    else:  # Fill with zeros when no transmitted
        newx += [0]
        newy += [0]
        newu += [0]
        newb += [0]
        newvel += [-1]
    newTimer += [(i+1)*ts]

xs = np.array(newx)
ys = np.array(newy)
bs = np.array(newb)
us = np.array(newu)
vs = np.array(newvel)
Time = np.array(newTimer)
#print(vs[0:10]*3.6)

transmitted positions 241
resampling points 667


## Save data

In [4]:
save_data_raw = pd.DataFrame()
save_data_raw['Timer'] = Timer
save_data_raw['xr'] = xr
save_data_raw['yr'] = yr
pickle.dump( save_data_raw, open( sol_file+"/save_data_raw.pkl", "wb" ) )

save_data = pd.DataFrame()
save_data['Time'] = Time
save_data['xs'] = xs
save_data['ys'] = ys

save_data['us'] = us
save_data['bs'] = bs
save_data['vs_m_s'] = vs
save_data['vs_km_h'] = vs*3.6

pickle.dump( save_data, open( sol_file+"/save_data.pkl", "wb" ) )

save_data_raw = pickle.load( open( sol_file+"/save_data_raw.pkl", "rb" ) )
save_data = pickle.load( open( sol_file+"/save_data.pkl", "rb" ) )

## Plot data

In [5]:
# Vector for plotting, removed the non-transmitted instants
Timep = Time[bs != 0]
xsp = xs[bs != 0] 
ysp = ys[bs != 0] 

fig = figure(plot_width=900, plot_height=450)
fig.line(Timer, xr, line_color='navy', legend_label="x", alpha=0.2)
fig.circle(Timer, xr, line_color='navy', legend_label="x", fill_alpha=0.2, alpha=0.2)
fig.line(Timep, xsp, line_color='navy', legend_label="x")
fig.circle(Timep, xsp, line_color='navy', color = 'cyan', legend_label="x")
fig.line(Timer, yr, line_color='red', legend_label="y", alpha=0.2)
fig.circle(Timer, yr, line_color='red', legend_label="y", fill_alpha=0.2, alpha=0.2)
fig.line(Timep, ysp, line_color='red', legend_label="y")
fig.circle(Timep, ysp, line_color='red', color = 'pink', legend_label="y")
fig.xaxis.axis_label = "time [s]"
fig.yaxis.axis_label = "y"
show(fig)

fig = figure(plot_width=900, plot_height=450)
fig.line(xsp, ysp, line_color='navy', legend_label="p")
fig.circle(xsp, ysp, color='navy', legend_label="p")
fig.line(xr, yr, line_color='navy', legend_label="p raw", alpha=0.2)
fig.circle(xr, yr, color='navy', legend_label="p raw", fill_alpha=0.2, alpha=0.2)
fig.xaxis.axis_label = "x"
fig.yaxis.axis_label = "y"
show(fig)

## Define MPC method privacy worst-case

In [6]:
# Create the dynamic system state
n = 30
X = np.array([0]*n)
Y = np.array([0]*n)
N = np.array([0]*n)

system = pd.DataFrame()
system['X'] = X
system['Y'] = Y
system['N'] = N

def system_update(system,x,y,b):
    dict = {'X': x, 'Y': y, 'N': b}
    system = system.append(dict, ignore_index = True).drop(index=[0]).reset_index(drop=True)
    return system

def privacy(system):
    n = system['N'].sum()
    if n == 0:
        barx = 0
        bary = 0
        priv = 0
    else:
        barx = np.dot(system['X'], system['N'])/n
        bary = np.dot(system['Y'], system['N'])/n
        #priv = np.dot(np.sqrt((system['X'] - barx)**2 + (system['Y'] - bary)**2), system['N'])/n 
        priv = np.dot((system['X'] - barx)**2 + (system['Y'] - bary)**2, system['N'])/n 
    return barx, bary, priv

def utility(x,y):
    return np.sqrt(x**2 + y**2)

def reset_system(system):
    n = len(system)
    X = np.array([0]*n)
    Y = np.array([0]*n)
    N = np.array([0]*n)

    system['X'] = X
    system['Y'] = Y
    system['N'] = N
    return system

In [7]:
from sys import path
from casadi import *
from pylab import *

nbuf = n
# Control
u = MX.sym("u")
v = MX.sym("v")
b = MX.sym("b")

# State
x = MX.sym("x",nbuf)
y = MX.sym("y",nbuf)
nx = MX.sym("nx",nbuf)

# Matrices
A = np.zeros((nbuf, nbuf))
B = np.zeros((nbuf, 1))
for i in range(nbuf-1):
    A[i,i+1] = 1

B[nbuf-1,0] = 1
A.tolist()
B.tolist()
A = DM(A)
B = DM(B)

# Dynamics
xplus = A@x + B@u
yplus = A@y + B@v
nplus = A@nx + B@b

# Discrete time dynamics function
F = Function('F', [x,u],[xplus])

#Privacity function
O = DM(np.ones((1,nbuf)))
xbar = (O@(x*nx))/((O@nx)+1e-8)
ybar = O@(y*nx)/((O@nx)+1e-8)
P = (O@( ((x-xbar)**2 + (y-ybar)**2)*nx) )/((O@nx)+1e-10)
J = Function('J', [x,y,nx],[xbar, ybar, P],['x','y','nx'],['xbar', 'ybar','P'])

# Test privacy functions
x1 = np.random.normal(loc=0.0, scale=0.35, size=(1,nbuf)).tolist()[0]
y1 = np.random.normal(loc=0.0, scale=0.35, size=(1,nbuf)).tolist()[0]
nx1 = [1]*nbuf

Jk = J(x=x1,y=y1,nx=nx1)
print(Jk)


{'P': DM(0.280316), 'xbar': DM(-0.00147551), 'ybar': DM(-0.0153318)}


In [8]:
system = reset_system(system)
barxk = []
baryk = []
privk = []

util = []
for i in range(0,nt,1):
    bi = bs[i] 
    # Update the real system
    system = system_update(system,xs[i],ys[i],bi)
    barx, bary, priv = privacy(system)
    barxk += [barx] 
    baryk += [bary] 
    privk += [priv] 


Data_real = pd.DataFrame()
Data_real['x'] = xs
Data_real['y'] = ys
Data_real['b'] = bs
Data_real['barx'] = barxk
Data_real['bary'] = baryk
Data_real['priv'] = privk
Data_real['u_enu'] = us

pickle.dump( Data_real, open( sol_file+"/Data_real_offline.pkl", "wb" ) )

from bokeh.plotting import figure, show, output_notebook, output_file, reset_output, save
from bokeh.layouts import gridplot, row, column
from bokeh.models import Range1d

fig = figure(plot_width=900, plot_height=350)
fig.line(Timep, Data_real.loc[Data_real.b>0].priv, line_color='black', legend_label="priv real")
fig.circle(Timep, Data_real.loc[Data_real.b>0].priv, color='black', legend_label="priv real")
show(fig)

In [9]:
## MPC method using worst-case privacy

def solve_mpc(xs,ys,bs,horizon,nbuf,util_bound):
    nt = len(xs)
    x_mpc = []
    y_mpc = []
    U_mpc = []
    V_mpc = []
    barx_mpc    = []
    bary_mpc    = []
    priv_mpc    = []
    priv_hp     = []
    priv_mpc_hp = []
    util_mpc    = []
    time_mpc    = []
    X0 = np.array([0]*nbuf).tolist()
    Y0 = np.array([0]*nbuf).tolist()
    N0 = np.array([0]*nbuf).tolist()

    system_opt = pd.DataFrame()
    system_opt['X'] = X0
    system_opt['Y'] = Y0
    system_opt['N'] = N0

    for tk in range(0,nt-horizon):#nt-horizon):

        if bs[tk] == 0:
            U_mpc += [0]
            V_mpc += [0]
            x_mpc += [xs[tk]]
            y_mpc += [ys[tk]]
            system_opt = system_update(system_opt, xs[tk], ys[tk], 0)
            barxt, baryt, privt = privacy(system_opt)
            barx_mpc += [barxt] 
            bary_mpc += [baryt] 
            priv_mpc += [privt]
            priv_hp  += [privt]
            priv_mpc_hp  += [privt]
            # Compute utility
            util_mpc += [-1]
            time_mpc += [0]
            #system_opt = system_update(system_opt, xs[tk], ys[tk], 0)

        else:
            time_0= time.time()
            X0 = system_opt['X'].tolist()
            Y0 = system_opt['Y'].tolist()
            N0 = system_opt['N'].tolist()

            # Initial conditions
            Jk = 0
            w=[]
            w0 = []
            lbw = []
            ubw = []
            g=[]
            lbg = []
            ubg = []

            # Constraints on input
            U = MX.sym("U",1)
            V = MX.sym("V",1)
            w += [U, V]
            lbw += [-inf]*2
            ubw += [inf]*2
            w0 += [0]*2
            g   += [U**2 + V**2]
            lbg += [0]
            ubg += [util_bound[tk]**2]

            # Initial conditions
            Xk = MX.sym("Xk",nbuf)
            w += [Xk]
            lbw += X0
            ubw += X0
            w0 += X0
            Yk = MX.sym("Yk",nbuf)
            w += [Yk]
            lbw += Y0
            ubw += Y0
            w0 += Y0
            Nk = MX.sym("Nk",nbuf)
            w += [Nk]
            lbw += N0
            ubw += N0
            w0 += N0

            # Future values of x, y and n
            ##Same position
            #Xf=xs[tk]*np.ones(horizon)
            #Yf=ys[tk]*np.ones(horizon)
            #Nf=bs[tk]*np.ones(horizon)
            ##
            
            ### minimizing privacy
            Xf=np.zeros(horizon)
            Yf=np.zeros(horizon)
            Nf=bs[tk]*np.ones(horizon)
            
            Xf[0]=xs[tk]
            Yf[0]=ys[tk]
            ###
            
            system_aux=system_opt.copy()
            system_aux=system_update(system_aux,Xf[0],Yf[0],Nf[0])
            for kk in range(1,horizon):
                Xf[kk],Yf[kk],p1=privacy(system_aux.iloc[1:]) #comment when using same position
                system_aux=system_update(system_aux,Xf[kk],Yf[kk],Nf[kk])
                
            
            bx,by,p1 =privacy(system_aux)
            priv_hp+=[p1]
            
            
            Xf = Xf[::].tolist()
            Xkfut = MX.sym("Xkfut",horizon) 
            w += [Xkfut]
            lbw += Xf
            ubw += Xf
            w0 += Xf
            Yf = Yf[::].tolist()
            Ykfut = MX.sym("Ykfut",horizon) 
            w += [Ykfut]
            lbw += Yf
            ubw += Yf
            w0 += Yf

            Nf = Nf[::].tolist()
            Nkfut = MX.sym("Nkfut",horizon) 
            w += [Nkfut]
            lbw += Nf
            ubw += Nf
            w0 += Nf

            # Apply the first input 
            Xk = F(Xk,Xkfut[0]+U)
            Yk = F(Yk,Ykfut[0]+V)
            Nk = F(Nk,1)
            Jt = J(x=Xk,y=Yk,nx=Nk)
            Jk += Jt['P']
            for k in range(horizon-1):
                # New NLP variable for the control
                Uk = MX.sym('U_' + str(k))
                Vk = MX.sym('V_' + str(k))
                w   += [Uk, Vk]
                lbw += [-util_bound[tk+k+1]*Nf[k+1]]*2
                ubw += [util_bound[tk+k+1]*Nf[k+1]]*2
                w0 += [0]*2

                g   += [Uk**2 + Vk**2]
                lbg += [0]
                ubg += [util_bound[tk+k+1]**2]

                Xk = F(Xk,Xkfut[k+1]+Uk)
                Yk = F(Yk,Ykfut[k+1]+Vk)
                Nk = F(Nk,Nkfut[k+1])
                Jt = J(x=Xk,y=Yk,nx=Nk)
                Jk += Jt['P']
            #Jk += Jt['P']

            prob = {'f': -Jk, 'x': vertcat(*w), 'g': vertcat(*g)}
            solver = nlpsol('solver', 'ipopt', prob);
            sol = solver(x0=w0, lbx=lbw, ubx=ubw, lbg=lbg, ubg=ubg)
            w_opt = sol['x'].full().flatten()
            J_opt = -sol['f']

            # Plot the solution
            u_opt = w_opt[0]
            v_opt = w_opt[1]
                            
            U_mpc += [u_opt]
            V_mpc += [v_opt]
            x_mpc += [Xf[0] + u_opt]
            y_mpc += [Yf[0] + v_opt]
            system_opt = system_update(system_opt, Xf[0] + u_opt, Yf[0] + v_opt, 1)
            barxt, baryt, privt = privacy(system_opt)
            barx_mpc += [barxt] 
            bary_mpc += [baryt] 
            priv_mpc += [privt] 
            # Compute utility
            util_mpc += [utility(u_opt,v_opt)]
            time_mpc += [time.time()-time_0]
            
            #last future privacy
            u_fut=np.zeros(horizon-1)
            v_fut=np.zeros(horizon-1)
            system_aux2=system_opt.copy()
            
            for k in range(horizon-1):
                u_fut[k]=w_opt[1+3*nbuf+3*horizon+k]
                v_fut[k]=w_opt[1+3*nbuf+3*horizon+k+1]
                system_aux2=system_update(system_aux2, Xf[k+1] + u_fut[k], Yf[k+1] + v_fut[k], 1)
            #system_opt = system_update(system_opt, Xf[0] + u_opt, Yf[0] + v_opt, 1)
            barx_fut, bary_fut, priv_fut = privacy(system_aux2)
            priv_mpc_hp += [priv_fut]
            
    clear_output(wait=False)

    data_mpc = pd.DataFrame()
    data_mpc['x'] = x_mpc
    data_mpc['y'] = y_mpc
    data_mpc['u_enu']= us[:len(x_mpc)]
    data_mpc['U'] = U_mpc
    data_mpc['V'] = V_mpc
    data_mpc['barx'] = barx_mpc
    data_mpc['bary'] = bary_mpc
    data_mpc['priv'] = priv_mpc
    data_mpc['priv_hp'] = priv_hp #privacy in i+hp worst case
    data_mpc['priv_mpc_hp'] = priv_mpc_hp #privacy in i+hp using mpc over worst case
    data_mpc['util'] = util_mpc
    data_mpc['time'] = time_mpc
    
    return data_mpc#U_mpc, V_mpc, x_mpc, y_mpc, barx_mpc, bary_mpc, priv_mpc, util_mpc 

In [10]:
start_time = time.monotonic()
h = Hp+1
data_mpc = solve_mpc(xs[:h+1],ys,bs,h,nbuf,100*np.ones(len(xs[:h+1])))
end_time = time.monotonic()
print(timedelta(seconds=end_time - start_time))
print(data_mpc['time'])


0:00:00.922000
0    0.828065
Name: time, dtype: float64


## Solve mpc privacy worst-case

In [11]:
start_time = time.monotonic()
Data_MPC = {} # Empty dictionary
util =[50, 100, 150, 200, 300, 400, 500,750,1000,1500,2000] #Upper bounds of utility loss function 
np.savetxt(sol_file+'/utility.txt',util)

sol_MPC=sol_file+'/sol_MPC_worstcase'
if not os.path.exists(sol_MPC):
    os.makedirs(sol_MPC)

for u in util:
    data_mpc = solve_mpc(xs,ys,bs,h,nbuf,u*np.ones(len(xs)))
    Data_MPC[u] = data_mpc
    end_time = time.monotonic()
    Data_MPC[u]['dif']=Data_MPC[u]['priv_mpc_hp']-Data_MPC[u]['priv_hp']
    Data_MPC[u]['dif_perc']=(Data_MPC[u]['priv_mpc_hp']-Data_MPC[u]['priv_hp'])/(Data_MPC[u]['priv_hp']+10**-10)
    Data_MPC[u]['dif_perc2']=(Data_MPC[u]['priv_mpc_hp']-Data_MPC[u]['priv_hp'])/(Data_MPC[u]['priv_mpc_hp']+10**-10)
    print(u,timedelta(seconds=end_time - start_time))
    Data_MPC[u].to_csv(sol_MPC+'/solmpc_'+str(Hp)+'_umax_'+str(u)+'.csv')


2000 2:19:29.875000


In [12]:
np.savetxt(sol_file+'utility.txt',util)


In [13]:
#Ploting privacy for a given bound Dj=u
from bokeh.plotting import figure, show, output_notebook, output_file, reset_output, save
from bokeh.layouts import gridplot, row, column

priv_hp = Data_MPC[u].priv_hp.loc[Data_MPC[u].util>=0]
priv_mpc_hp = Data_MPC[u].priv_mpc_hp.loc[Data_MPC[u].util>=0]

#priv_wc = df.priv_hp
#priv_mpc = Data_MPC[u].priv_hp
Timep2 = Time[0:len(xs)-h][bs[0:len(xs)-h] != 0]
fig = figure(plot_width=900, plot_height=250)
#fig.line(np.arange(len(xs)-Hp), Priv_Hp, line_color='navy', legend_label="priv worst case")
#fig.circle(np.arange(len(xs)-Hp), Priv_Hp, color='navy', legend_label="priv worst case")
fig.line(Timep2, priv_hp, line_color='navy', legend_label="priv worst case")
fig.circle(Timep2, priv_hp, color='navy', legend_label="priv worst case")
fig.line(Timep2, priv_mpc_hp, line_color='orange', legend_label="priv MPC worst case")
fig.circle(Timep2, priv_mpc_hp, color='orange', legend_label="priv MPC worst case")
# fig.circle(Time, bs, color='green', legend_label="transm")
fig.xaxis.axis_label = "time [s]"
fig.yaxis.axis_label = "privacy"
show(fig)


## Plot $f_h$ function

In [14]:
gain=pd.DataFrame(index=util)
gain.index.name='utility'
gain['min']=[Data_MPC[u].dif.loc[Data_MPC[u].util>0].min() for u in util]
gain['max']=[Data_MPC[u].dif.loc[Data_MPC[u].util>0].max() for u in util]
gain['mean']=[Data_MPC[u].dif.loc[Data_MPC[u].util>0].mean() for u in util]
gain['p_05']=[Data_MPC[u].dif.loc[Data_MPC[u].util>0].quantile(q=0.05) for u in util]
gain['p_95']=[Data_MPC[u].dif.loc[Data_MPC[u].util>0].quantile(q=0.95) for u in util]

new_row = pd.Series(data={'min':0, 'max':0, 'mean':0,'p_05':0,'p_95':0}, name=0)
gain = gain.append(new_row, ignore_index=False)
gain=gain.sort_index()

#gain.to_csv(sol_file+'/gain_'+str(Hp)+'.csv')

fig = figure(plot_width=900, plot_height=250)
fig.circle(gain.index, gain['p_05'], color='red', legend_label="bound")
fig.line(gain.index, gain['p_05'], color='red', legend_label="bound")
fig.circle(gain.index, gain['p_95'], color='red', legend_label="bound")
fig.line(gain.index, gain['p_95'], color='red', legend_label="bound")
fig.circle(gain.index, gain['mean'], color='black', legend_label="mean")
fig.line(gain.index, gain['mean'], color='black', legend_label="mean")

fig.xaxis.axis_label = "utility"
fig.yaxis.axis_label = "absolute difference privacy"
show(fig)
